In [1]:
%matplotlib inline

import gym
from gym.wrappers import Monitor
import itertools
import numpy as np
import os
import random
import sys
import psutil
import tensorflow as tf

if "../" not in sys.path:
  sys.path.append("../")

from lib.tile_coding import IHT, tiles
from lib import plotting
from collections import deque, namedtuple

/Users/michaeloneill/Documents/RL/venv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
env = gym.make("MountainCar-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
env.action_space

Discrete(3)

In [4]:
class Estimator():
    """Q-Value Estimator neural network.

    This network is used for both the Q-Network and the Target Network.
    """

    def __init__(self, scope="estimator", summaries_dir=None, num_tilings=8, max_size=4096):
        self.scope = scope
        self.max_size = max_size
        self.num_tilings = num_tilings

        self.iht = IHT(max_size)
        
         # position and velocity needs scaling to satisfy the tile software
        self.position_scale = self.num_tilings / (env.observation_space.high[0] - env.observation_space.low[0])
        self.velocity_scale = self.num_tilings / (env.observation_space.high[1] - env.observation_space.low[1])
        
        
        # Writes Tensorboard summaries to disk
        self.summary_writer = None
        with tf.variable_scope(scope):
            # Build the graph
            self._build_model()
            if summaries_dir:
                summary_dir = os.path.join(summaries_dir, "summaries_{}".format(scope))
                if not os.path.exists(summary_dir):
                    os.makedirs(summary_dir)
                self.summary_writer = tf.summary.FileWriter(summary_dir)

    def featurize_state_action(self, state, action):
        """
        Returns the featurized representation for a state.
        """
        featurized = tiles(self.iht, self.num_tilings, [self.position_scale * state[0], self.velocity_scale * state[1]], [action])
       
        return featurized
    
    def _build_model(self):
        """
        Builds the Tensorflow graph.
        """
        # Placeholders for our input
        # Our input are 4 RGB frames of shape 160, 160 each
        self.X_pl = tf.placeholder(shape=[None, self.num_tilings], dtype=tf.float32, name="X")
        # The TD target value
        self.y_pl = tf.placeholder(shape=[None], dtype=tf.float32, name="y")
        # Integer id of which action was selected
        self.actions_pl = tf.placeholder(shape=[None], dtype=tf.int32, name="actions")

        #X = tf.to_float(self.X_pl) / 255.0
        batch_size = tf.floordiv(tf.shape(self.X_pl)[0], 3)
        
        fc1 = tf.contrib.layers.fully_connected(self.X_pl, 100)
        self.predictions = tf.contrib.layers.fully_connected(
            fc1, 1, activation_fn=None)

        # Get the predictions for the chosen actions only
        
        gather_indices = tf.range(batch_size) * 3 + self.actions_pl
        self.action_predictions = tf.squeeze(tf.gather(self.predictions, gather_indices), axis=1)

        # Calculate the loss
        self.losses = tf.squared_difference(self.y_pl, self.action_predictions)
        self.loss = tf.reduce_mean(self.losses)

        # Optimizer Parameters from original paper
        self.optimizer = tf.train.GradientDescentOptimizer(0.001)
        #self.optimizer = tf.train.RMSPropOptimizer(0.00025, 0.99, 0.0, 1e-6)
        self.train_op = self.optimizer.minimize(self.loss, global_step=tf.contrib.framework.get_global_step())

        # Summaries for Tensorboard
        self.summaries = tf.summary.merge([
            tf.summary.scalar("loss", self.loss),
            tf.summary.histogram("loss_hist", self.losses),
            tf.summary.histogram("q_values_hist", self.predictions),
            tf.summary.scalar("max_q_value", tf.reduce_max(self.predictions))
        ])

    def predict(self, sess, s, a=None):
        """
        Predicts action values.

        Args:
          sess: Tensorflow session
          s: State input of shape [batch_size, 4, 160, 160, 3]

        Returns:
          Tensor of shape [batch_size, NUM_VALID_ACTIONS] containing the estimated 
          action values.
        """
        features = []
        if a is None:
            for s_ in s:
                features.extend([self.featurize_state_action(s_, i) for i in range(env.action_space.n)])
        else:
            raise ValueError
            for s_ in s:
                features.extend([self.featurize_state_action(s, a)])
        return sess.run(self.predictions, {self.X_pl: np.array(features)})

    def update(self, sess, s, a, y):
        """
        Updates the estimator towards the given targets.

        Args:
          sess: Tensorflow session object
          s: State input of shape [batch_size, 4, 160, 160, 3]
          a: Chosen actions of shape [batch_size]
          y: Targets of shape [batch_size]

        Returns:
          The calculated loss on the batch.
        """
     
        features = []
        for s_ in s:
            features.extend([self.featurize_state_action(s_, i) for i in range(env.action_space.n)])
        
        feed_dict = { self.X_pl: features, self.y_pl: y, self.actions_pl: a}
        summaries, global_step, _, loss = sess.run(
            [self.summaries, tf.contrib.framework.get_global_step(), self.train_op, self.loss],
            feed_dict)
        if self.summary_writer:
            self.summary_writer.add_summary(summaries, global_step)
        return loss

In [5]:
# For Testing....

tf.reset_default_graph()
global_step = tf.Variable(0, name="global_step", trainable=False)

e = Estimator(scope="test")

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Example observation batch
    observation = env.reset()
    
    #observation_p = sp.process(sess, observation)
    observations = np.array([observation] * 2)
    
    # Test Prediction
    print(e.predict(sess, observations))

    # Test training step
    y = np.array([10, 10])
    a = np.array([0, 1])
    print(e.update(sess, observations, a, y))

Instructions for updating:
Please switch to tf.train.get_global_step
[[-0.9678454]
 [-4.6341457]
 [-8.2629175]
 [-0.9678454]
 [-4.6341457]
 [-8.2629175]]
167.22592


/Users/michaeloneill/Documents/RL/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
class ModelParametersCopier():
    """
    Copy model parameters of one estimator to another.
    """
    
    def __init__(self, estimator1, estimator2):
        """
        Defines copy-work operation graph.  
        Args:
          estimator1: Estimator to copy the paramters from
          estimator2: Estimator to copy the parameters to
        """
        e1_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator1.scope)]
        e1_params = sorted(e1_params, key=lambda v: v.name)
        e2_params = [t for t in tf.trainable_variables() if t.name.startswith(estimator2.scope)]
        e2_params = sorted(e2_params, key=lambda v: v.name)

        self.update_ops = []
        for e1_v, e2_v in zip(e1_params, e2_params):
            op = e2_v.assign(e1_v)
            self.update_ops.append(op)
            
    def make(self, sess):
        """
        Makes copy.
        Args:
            sess: Tensorflow session instance
        """
        sess.run(self.update_ops)

In [7]:
def make_epsilon_greedy_policy(estimator, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function approximator and epsilon.

    Args:
        estimator: An estimator that returns q values for a given state
        nA: Number of actions in the environment.

    Returns:
        A function that takes the (sess, observation, epsilon) as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.

    """
    def policy_fn(sess, observation, epsilon):
        A = np.ones(nA, dtype=float) * epsilon / nA
        q_values = estimator.predict(sess, np.expand_dims(observation, 0))
        best_action = np.argmax(q_values)
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [8]:
def deep_q_learning(sess,
                    env,
                    q_estimator,
                    target_estimator,
                    state_processor,
                    num_episodes,
                    experiment_dir,
                    replay_memory_size=500000,
                    replay_memory_init_size=50000,
                    update_target_estimator_every=10000,
                    discount_factor=0.9,
                    epsilon_start=1,
                    epsilon_end=0.1,
                    epsilon_decay_steps=500000,
                    batch_size=50,
                    record_video_every=50):
    """
    Q-Learning algorithm for off-policy TD control using Function Approximation.
    Finds the optimal greedy policy while following an epsilon-greedy policy.

    Args:
        sess: Tensorflow Session object
        env: OpenAI environment
        q_estimator: Estimator object used for the q values
        target_estimator: Estimator object used for the targets
        state_processor: A StateProcessor object
        num_episodes: Number of episodes to run for
        experiment_dir: Directory to save Tensorflow summaries in
        replay_memory_size: Size of the replay memory
        replay_memory_init_size: Number of random experiences to sampel when initializing 
          the reply memory.
        update_target_estimator_every: Copy parameters from the Q estimator to the 
          target estimator every N steps
        discount_factor: Gamma discount factor
        epsilon_start: Chance to sample a random action when taking an action.
          Epsilon is decayed over time and this is the start value
        epsilon_end: The final minimum value of epsilon after decaying is done
        epsilon_decay_steps: Number of steps to decay epsilon over
        batch_size: Size of batches to sample from the replay memory
        record_video_every: Record a video every N episodes

    Returns:
        An EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """

    Transition = namedtuple("Transition", ["state", "action", "reward", "next_state", "done"])

    # The replay memory
    replay_memory = []
    
    # Make model copier object
    estimator_copy = ModelParametersCopier(q_estimator, target_estimator)

    # Keeps track of useful statistics
    stats = plotting.EpisodeStats(
        episode_lengths=np.zeros(num_episodes),
        episode_rewards=np.zeros(num_episodes))
    
    # For 'system/' summaries, usefull to check if currrent process looks healthy
    current_process = psutil.Process()

    # Create directories for checkpoints and summaries
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    checkpoint_path = os.path.join(checkpoint_dir, "model")
    monitor_path = os.path.join(experiment_dir, "monitor")
    
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    if not os.path.exists(monitor_path):
        os.makedirs(monitor_path)

    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    #latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    #if latest_checkpoint:
    #    print("Loading model checkpoint {}...\n".format(latest_checkpoint))
    #    saver.restore(sess, latest_checkpoint)
    
    # Get the current time step
    total_t = sess.run(tf.contrib.framework.get_global_step())

    # The epsilon decay schedule
    epsilons = np.linspace(epsilon_start, epsilon_end, epsilon_decay_steps)

    # The policy we're following
    policy = make_epsilon_greedy_policy(
        q_estimator, 3)

    # Populate the replay memory with initial experience
    print("Populating replay memory...")
    state = env.reset()
    #state = state_processor.process(sess, state)
    #state = np.stack([state] * 4, axis=2)
    for i in range(replay_memory_init_size):
        action_probs = policy(sess, state, epsilons[min(total_t, epsilon_decay_steps-1)])
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
        next_state, reward, done, _ = env.step(action)
       
        #next_state = state_processor.process(sess, next_state)
        #next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)
        replay_memory.append(Transition(state, action, reward, next_state, done))
        if done:
            state = env.reset()
            #state = state_processor.process(sess, state)
            #state = np.stack([state] * 4, axis=2)
        else:
            state = next_state


    # Record videos
    # Add env Monitor wrapper
    #env = Monitor(env, directory=monitor_path, video_callable=lambda count: count % record_video_every == 0, resume=True)

    for i_episode in range(num_episodes):

        # Save the current checkpoint
        saver.save(tf.get_default_session(), checkpoint_path)

        # Reset the environment
        state = env.reset()
        #state = state_processor.process(sess, state)
        #state = np.stack([state] * 4, axis=2)
        loss = None

        # One step in the environment
        for t in itertools.count():

            # Epsilon for this time step
            epsilon = epsilons[min(total_t, epsilon_decay_steps-1)]

            # Maybe update the target estimator
            if total_t % update_target_estimator_every == 0:
                estimator_copy.make(sess)
                print("\nCopied model parameters to target network.")

            # Take a step
            action_probs = policy(sess, state, epsilon)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(action)
            
            #next_state = state_processor.process(sess, next_state)
            #next_state = np.append(state[:,:,1:], np.expand_dims(next_state, 2), axis=2)

            # If our replay memory is full, pop the first element
            if len(replay_memory) == replay_memory_size:
                replay_memory.pop(0)

            # Save transition to replay memory
            replay_memory.append(Transition(state, action, reward, next_state, done))   

            # Update statistics
            stats.episode_rewards[i_episode] += reward
            stats.episode_lengths[i_episode] = t

            # Sample a minibatch from the replay memory
            samples = random.sample(replay_memory, batch_size)
            states_batch, action_batch, reward_batch, next_states_batch, done_batch = map(np.array, zip(*samples))
         
            # Calculate q values and targets
            q_values_next = target_estimator.predict(sess, next_states_batch)  
            q_values_next = q_values_next.reshape((-1, 3))
            
            targets_batch = reward_batch + np.invert(done_batch).astype(np.float32) * discount_factor * np.amax(q_values_next, axis=1)

            # Perform gradient descent update
            #states_batch = np.array(states_batch)
            loss = q_estimator.update(sess, states_batch, action_batch, targets_batch)

            if done:
                break

            state = next_state
            total_t += 1
            
        
        # Print out which step we're on, useful for debugging.
        sys.stdout.write("\rEpisode {}/{}, loss: {}, reward: {}, epsilon: {}".format(
                i_episode + 1, num_episodes, loss, stats.episode_rewards[i_episode], epsilon))
        sys.stdout.flush()

        # Add summaries to tensorboard
        episode_summary = tf.Summary()
        episode_summary.value.add(simple_value=epsilon, tag="episode/epsilon")
        episode_summary.value.add(simple_value=stats.episode_rewards[i_episode], tag="episode/reward")
        episode_summary.value.add(simple_value=stats.episode_lengths[i_episode], tag="episode/length")
        episode_summary.value.add(simple_value=current_process.cpu_percent(), tag="system/cpu_usage_percent")
        episode_summary.value.add(simple_value=current_process.memory_percent(memtype="vms"), tag="system/v_memeory_usage_percent")
        q_estimator.summary_writer.add_summary(episode_summary, i_episode)
        q_estimator.summary_writer.flush()
        
        yield total_t, plotting.EpisodeStats(
            episode_lengths=stats.episode_lengths[:i_episode+1],
            episode_rewards=stats.episode_rewards[:i_episode+1])

    return stats

In [9]:
tf.reset_default_graph()

# Where we save our checkpoints and graphs
experiment_dir = os.path.abspath("./experiments/{}".format(env.spec.id))

# Create a glboal step variable
global_step = tf.Variable(0, name='global_step', trainable=False)
    
# Create estimators
q_estimator = Estimator(scope="q_estimator", summaries_dir=experiment_dir)
target_estimator = Estimator(scope="target_q")

# State processor
#state_processor = StateProcessor()

# Run it!
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t, stats in deep_q_learning(sess,
                                    env,
                                    q_estimator=q_estimator,
                                    target_estimator=target_estimator,
                                    state_processor=None,
                                    experiment_dir=experiment_dir,
                                    num_episodes=10000,
                                    replay_memory_size=500000,
                                    replay_memory_init_size=50000,
                                    update_target_estimator_every=10000,
                                    epsilon_start=1.0,
                                    epsilon_end=0.1,
                                    epsilon_decay_steps=500000,
                                    discount_factor=0.9,
                                    batch_size=50):
        pass

            #print("\nEpisode Reward: {}".format(stats.episode_rewards[-1]))

/Users/michaeloneill/Documents/RL/venv/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Populating replay memory...

Copied model parameters to target network.
Episode 50/10000, loss: 122.32513427734375, reward: -200.0, epsilon: 0.9820899641799283
Copied model parameters to target network.
Episode 100/10000, loss: 5.799839496612549, reward: -200.0, epsilon: 0.9641799283598567
Copied model parameters to target network.
Episode 150/10000, loss: 2.7823326587677, reward: -200.0, epsilon: 0.9462698925397851
Copied model parameters to target network.
Episode 201/10000, loss: 0.13712400197982788, reward: -200.0, epsilon: 0.928001656003312
Copied model parameters to target network.
Episode 251/10000, loss: 0.11807287484407425, reward: -200.0, epsilon: 0.9100916201832404
Copied model parameters to target network.
Episode 301/10000, loss: 3.968113660812378, reward: -200.0, epsilon: 0.8921815843631687
Copied model parameters to target network.
Episode 351/10000, loss: 3.379532814025879, reward: -200.0, epsilon: 0.8742715485430971
Copied model parameters to target network.
Episode 40

KeyboardInterrupt: 

In [16]:
import time
env = gym.make("MountainCar-v0")
epsilon = 0
#for i_episide in range(20):
state = env.reset()
with tf.Session() as sess:
    # The policy we're following
    policy = make_epsilon_greedy_policy(
        q_estimator, env.action_space.n)
    saver = tf.train.Saver()
    # Load a previous checkpoint if we find one
    experiment_dir = os.path.abspath("./experiments/{}".format(env.spec.id))
    checkpoint_dir = os.path.join(experiment_dir, "checkpoints")
    latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_checkpoint:
        print("Loading model checkpoint {}...\n".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)
    for t in itertools.count():
        #env.render()
        time.sleep(0.01)
        action_probs = policy(sess, state, epsilon)
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
        state, _, done, _ = env.step(action)
        env.render()
        if done:
            break

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Loading model checkpoint /Users/michaeloneill/Documents/RL/wip/experiments/MountainCar-v0/checkpoints/model...

INFO:tensorflow:Restoring parameters from /Users/michaeloneill/Documents/RL/wip/experiments/MountainCar-v0/checkpoints/model


In [17]:
stats

Stats(episode_lengths=array([199., 199., 199., ..., 199., 199., 199.]), episode_rewards=array([-200., -200., -200., ..., -200., -200., -200.]))

In [18]:
len(stats[0])

3359

In [19]:
np.histogram(stats[1])

(array([   0,    0,    0,    0,    0, 3359,    0,    0,    0,    0]),
 array([-200.5, -200.4, -200.3, -200.2, -200.1, -200. , -199.9, -199.8,
        -199.7, -199.6, -199.5]))

In [20]:
np.histogram(stats[0])

(array([   0,    0,    0,    0,    0, 3359,    0,    0,    0,    0]),
 array([198.5, 198.6, 198.7, 198.8, 198.9, 199. , 199.1, 199.2, 199.3,
        199.4, 199.5]))

In [20]:
np.histogram(stats.episode_rewards)

(array([4532,   25,   19,   28,   23,   23,   14,    3,    4,    3]),
 array([-200. , -192.8, -185.6, -178.4, -171.2, -164. , -156.8, -149.6,
        -142.4, -135.2, -128. ]))